In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from filterpy.kalman import UnscentedKalmanFilter
from filterpy.kalman.sigma_points import MerweScaledSigmaPoints
from filterpy.kalman import JulierSigmaPoints
from filterpy.common import Q_discrete_white_noise
import random

In [2]:
def bbusppg(num):     # Returns B-bus..

    linedata = linedatas(num);
    fb = linedata[:,0];  #first column 
    tb = linedata[:,1];  #2nd column 
    b = linedata[:,4];   #5th column
    nbus = int(np.max([np.max(fb),np.max(tb)]))    # no. of buses...
    nbranch = len(fb);           # no. of branches...
    bbus = np.mat(np.zeros((nbus,nbus)));

    for k in range (0,nbranch):
        bbus[int(fb[k]-1),int(tb[k]-1)] = b[k]
        bbus[int(tb[k]-1),int(fb[k]-1)] = bbus[int(fb[k]-1),int(tb[k]-1)];

    return bbus
# Traditional Measurement Data..
# Vi - 1, Pi - 2, Qi - 3, Pij - 4, Qij - 5, Iij - 6],

In [3]:

def zdatas(i):

    Pow_i= Pi[i]; 


    #           |Msnt |Type | Value | From |   To |   Rii | 
    zdata7= np.mat([ #---- Voltage Magnitude ------------#
                [1,    1,    (Pow_i[0]/13.2)*1.732,   1,       0,   9e-4],
                # #-----------------------------------#
                # #---- Real Power Injection ---------#
                [2,    2,    Pow_i[1]/10,   3,       0,   1e-4],
                [3,    2,    Pow_i[2]/10,   5,       0,   1e-4], 
                [4,    2,    Pow_i[3]/10,   7,       0,   1e-4],
                #------------------------------------#
                #---- Reative Power Injection -------#
                [5,     3,    Pow_i[4]/10,   3,       0,   1e-4],
                [6,     3,    Pow_i[5]/10,   5,       0,   1e-4], 
                [7,     3,    Pow_i[6]/10,   7,       0,   1e-4],
                #------------------------------------#
                #------ Real Power Flow ------------- #
                [8,     4,   Pow_i[7]/10,   1,       2,   64e-6],
                [9,     4,   Pow_i[8]/10,   2,       3,   64e-6],
                [10,     4,  Pow_i[9]/10,   3,       4,   64e-6],
                [11,     4,  Pow_i[10]/10,   3,       7,   64e-6],
                [12,     4,  Pow_i[11]/10,   2,       6,   64e-6],
                [13,     4,  Pow_i[12]/10,   6,       5,   64e-6],
                [14,     4,  Pow_i[13]/10,   7,       6,   64e-6],
                #------------------------------------#
                #------ Reactive Power Flow ---------#   
                [15,     5,  Pow_i[14]/10,   1,       2,   64e-6],
                [16,     5,  Pow_i[15]/10,   2,       3,   64e-6],
                [17,     5,  Pow_i[16]/10,   3,       4,   64e-6],
                [18,     5,  Pow_i[17]/10,   3,       7,   64e-6],
                [19,     5,  Pow_i[18]/10,   2,       6,   64e-6],
                [20,     5,  Pow_i[19]/10,   6,       5,   64e-6],
                [21,     5,  Pow_i[20]/10,   7,       6,   64e-6]])
    return zdata7

In [4]:
# Line data function

def  linedatas(num):
    ## 
    #         |  From |  To   |   R     |   X     |     B/2  |  X'mer  |
    #         |  Bus  | Bus   |  pu     |  pu     |     pu   | TAP (a) |
    linedata7=np.mat([  [1,      2,       0.07149,   0.17840,       0.0,       1],
                        [2,      3,       0.00596,   0.01487,       0.0,       1],
                        [2,      6,       0.01291,   0.03221,       0.0,       1],
                        [3,      4,       0.02284,   0.05699,       0.0,       1],
                        [3,      7,       0.00496,   0.01239,       0.0,       1],
                        [6,      5,       0.01688,   0.04209,       0.0,       1],
                        [6,      7,       0.01390,   0.03469,       0.0,       1]])
    
    return linedata7

In [5]:
# Returns ybus
def ybusppg(num):  

    linedata = linedatas(num); # Calling "linedata6.m" for Line Data...
    fb = linedata[:,0];     # From bus number... i.e in the column 1
    tb = linedata[:,1];     # To bus number...
    r = linedata[:,2];      # Resistance, R...
    x = linedata[:,3];      # Reactance, X...
    b = linedata[:,4];      # Ground Admittance, B/2...
    a = linedata[:,5];      # Tap setting value..
    z = r + x*1j;            # Z matrix...
    y = 1/z;               # To get inverse of each element...
    b = b*1j;                # Make B imaginary...

    nbus = int(np.max([np.max(fb),np.max(tb)]));    # no. of buses...(14 in this case)
    nbranch = len(fb) # no. of branches...(= 20)
    ybus = np.mat(np.zeros((nbus,nbus), dtype=complex));       # Initialise YBus...
    
    # Formation of the Off Diagonal Elements...
    for k in range (0,nbranch):
        ybus[int(fb[k]-1),int(tb[k]-1)] =ybus[int(fb[k]-1),int(tb[k]-1)]- (y[k]/a[k]);
        ybus[int(tb[k]-1),int(fb[k]-1)] = ybus[int(fb[k]-1),int(tb[k]-1)];
 
 
    # Formation of Diagonal Elements....
    for m in range (0,nbus):
        for n in range (0,nbranch):
            if fb[n]-1 == m:
                ybus[m,m]=ybus[m,m]+ (y[n]/(a[n]**2)) + b[n]
            elif tb[n]-1 ==m:
                ybus[m,m]=ybus[m,m]+y[n]
                

    return ybus

In [6]:

# here we get the 27*41 matrix
def HJacobian_at1(x):
    num=7;
    del_T=0.001;
    ybus = ybusppg(num); # Get YBus..
    zdata = zdatas(1); # Get Measurement data..
    
    #in every iteration we take a new row to consider from the csv file we imported
    V=x[6:]
    del1 = np.zeros(num);
    del1[1:]=x[0:6]
    
    bpq = bbusppg(num); # Get B data..
    nbus = int(np.max([np.max(zdata[:,3]),np.max(zdata[:,4])])); # Get number of buses..
    type1 = zdata[:,1]; # Type of measurement, Vi - 1, Pi - 2, Qi - 3, Pij - 4, Qij - 5, Iij - 6..
#     z = zdata[:,2]; # Measuement values..
    fbus = zdata[:,3]; # From bus..
    tbus = zdata[:,4]; # To bus..
    Ri = np.diagflat(zdata[:,5]) # Measurement Error..
   
    G = ybus.real;
    B = ybus.imag 
    E = np.matrix(np.concatenate((del1[1:], V)))# E is the state matrix of 27*1

    vi = np.mat((type1 == 1).nonzero())[0,:]; # Index of measurements..
    ppi = np.mat((type1 == 2).nonzero())[0,:];
    qi = np.mat((type1 == 3).nonzero())[0,:]
    pf = np.mat((type1 == 4).nonzero())[0,:]
    qf = np.mat((type1 == 5).nonzero())[0,:]

    nvi = vi.shape[1]; # Number of Voltage measurements..
    npi = ppi.shape[1]; # Number of Real Power Injection measurements..
    nqi = qi.shape[1]; # Number of Reactive Power Injection measurements..
    npf = pf.shape[1]; # Number of Real Power Flow measurements..
    nqf = qf.shape[1]; # Number of Reactive Power Flow measurements..
    H11=np.mat(np.zeros((nvi,nbus-1)))

    H12=np.mat(np.zeros((nvi,nbus)))
    for k in range(0,nvi):
        for n in range(0,nvi):
            if n == k:
                 H12[k,n]=1

    H21=np.mat(np.zeros((npi,nbus-1)))
    for i in range(0,npi):
        m = int(fbus[(ppi[0,i])])-1
        for k in range(0,nbus-1):
            if k+1 == m:
                for n in range(0,nbus):
                    H21[i,k] += V[m]*V[n]*(-G[m,n]*np.sin(del1[m]-del1[n]) + B[m,n]*np.cos(del1[m]-del1[n]))
                H21[i,k] -= (V[m]**2)*B[m,m]
            else:
                H21[i,k]=V[m]*V[k+1]*(G[m,k+1]*np.sin(del1[m]-del1[k+1]) - B[m,k+1]*np.cos(del1[m]-del1[k+1]))

    H22=np.mat(np.zeros((npi,nbus)))
    for i in range(0,npi):
        m = int(fbus[(ppi[0,i])])-1
        for k in range(0,nbus):
            if k == m:
                for n in range(0,nbus):
                    H22[i,k] += V[n]*(G[m,n]*np.cos(del1[m]-del1[n]) + B[m,n]*np.sin(del1[m]-del1[n]))
                H22[i,k] += V[m]*G[m,m]
            else:
                H22[i,k]=V[m]*(G[m,k]*np.cos(del1[m]-del1[k]) + B[m,k]*np.sin(del1[m]-del1[k]))

    H31=np.mat(np.zeros((nqi,nbus-1)))
    for i in range(0,nqi):
        m = int(fbus[(qi[0,i])])-1
        for k in range(0,nbus-1):
            if k+1 == m:
                for n in range(0,nbus):
                     H31[i,k] += V[m]*V[n]*(G[m,n]*np.cos(del1[m]-del1[n]) + B[m,n]*np.sin(del1[m]-del1[n]))
                H31[i,k] -= (V[m]**2)*G[m,m]
            else:
                H31[i,k]=V[m]*V[k+1]*(-G[m,k+1]*np.cos(del1[m]-del1[k+1]) - B[m,k+1]*np.sin(del1[m]-del1[k+1]))

    H32=np.mat(np.zeros((nqi,nbus)))
    for i in range(0,nqi):
        m = int(fbus[(qi[0,i])])-1
        for k in range(0,nbus):
            if k == m:
                for n in range(0,nbus):
                     H32[i,k] += V[n]*(G[m,n]*np.sin(del1[m]-del1[n]) - B[m,n]*np.cos(del1[m]-del1[n]))
                H32[i,k] -= V[m]*B[m,m]
            else:
                H32[i,k]=V[m]*(G[m,k]*np.sin(del1[m]-del1[k]) - B[m,k]*np.cos(del1[m]-del1[k]))

    H41=np.mat(np.zeros((npf,nbus-1)))
    for i in range(0,npf):
        m = int(fbus[(pf[0,i])])-1
        n = int(tbus[(pf[0,i])])-1
        for k in range(0,nbus-1):
            if k+1 == m:
                H41[i,k] = V[m]*V[n]*(-G[m,n]*np.sin(del1[m]-del1[n]) + B[m,n]*np.cos(del1[m]-del1[n]))
            elif k+1 == n:
                H41[i,k] = -V[m]*V[n]*(-G[m,n]*np.sin(del1[m]-del1[n]) + B[m,n]*np.cos(del1[m]-del1[n]))
            else:
                H41[i,k] = 0

    H42=np.mat(np.zeros((npf,nbus)))
    for i in range(0,npf):
        m = int(fbus[(pf[0,i])])-1
        n = int(tbus[(pf[0,i])])-1
        for k in range(0,nbus):
            if k == m:
                H42[i,k] = -V[n]*(-G[m,n]*np.cos(del1[m]-del1[n]) - B[m,n]*np.sin(del1[m]-del1[n])) -2*G[m,n]*V[m]
            elif k == n:
                H42[i,k] = -V[m]*(-G[m,n]*np.cos(del1[m]-del1[n]) - B[m,n]*np.sin(del1[m]-del1[n]))
            else:
                H42[i,k] = 0

    H51=np.mat(np.zeros((nqf,nbus-1)))
    for i in range(0,nqf):
        m = int(fbus[(qf[0,i])])-1
        n = int(tbus[(qf[0,i])])-1
        for k in range(0,nbus-1):
            if k+1 == m:
                 H51[i,k] = -V[m]*V[n]*(-G[m,n]*np.cos(del1[m]-del1[n]) - B[m,n]*np.sin(del1[m]-del1[n]))
            elif k+1 == n:
                H51[i,k] = V[m]*V[n]*(-G[m,n]*np.cos(del1[m]-del1[n]) - B[m,n]*np.sin(del1[m]-del1[n]))
            else:
                H51[i,k] = 0

    H52=np.mat(np.zeros((nqf,nbus)))
    for i in range(0,nqf):
        m = int(fbus[(qf[0,i])])-1
        n = int(tbus[(qf[0,i])])-1
        for k in range(0,nbus):
            if k == m:
                H52[i,k] = -V[n]*(-G[m,n]*np.sin(del1[m]-del1[n]) + B[m,n]*np.cos(del1[m]-del1[n])) -2*V[m]*(-B[m,n]+bpq[m,n])
            elif k == n:
                H52[i,k] = -V[m]*(-G[m,n]*np.sin(del1[m]-del1[n]) + B[m,n]*np.cos(del1[m]-del1[n]))
            else:
                H52[i,k] = 0

    H1=np.mat(np.concatenate((H11,H12),axis=1))
    H2=np.mat(np.concatenate((H21,H22),axis=1))
    H3=np.mat(np.concatenate((H31,H32),axis=1))
    H4=np.mat(np.concatenate((H41,H42),axis=1))
    H5=np.mat(np.concatenate((H51,H52),axis=1))

    H=np.mat(np.concatenate((H1,H2,H3,H4,H5)))
    return H
    

In [16]:
def Hx(x):

    num=7;
    del_T=0.001;
    ybus = ybusppg(num); # Get YBus..
    zdata = zdatas(1); # Get Measurement data..
    
    #in every iteration we take a new row to consider from the csv file we imported

    V=x[6:]
    del1 = np.zeros(num);
    del1[1:]=x[0:6]
    bpq = bbusppg(num); # Get B data..
    nbus = int(np.max([np.max(zdata[:,3]),np.max(zdata[:,4])])); # Get number of buses..
    type1 = zdata[:,1]; # Type of measurement, Vi - 1, Pi - 2, Qi - 3, Pij - 4, Qij - 5, Iij - 6..
    z = zdata[:,2]; # Measuement values..
    fbus = zdata[:,3]; # From bus..
    tbus = zdata[:,4]; # To bus..
    Ri = np.diagflat(zdata[:,5]) # Measurement Error..
   
    G = ybus.real;
    B = ybus.imag 
    E = np.matrix(np.concatenate((del1[1:], V)))# E is the state matrix of 27*1

    vi = np.mat((type1 == 1).nonzero())[0,:] # Index of measurements..
    ppi =np.mat((type1 == 2).nonzero())[0,:]
    qi = np.mat((type1 == 3).nonzero())[0,:]
    pf = np.mat((type1 == 4).nonzero())[0,:]
    qf = np.mat((type1 == 5).nonzero())[0,:]

    nvi = vi.shape[1]; # Number of Voltage measurements..
    npi = ppi.shape[1]; # Number of Real Power Injection measurements..
    nqi = qi.shape[1]; # Number of Reactive Power Injection measurements..
    npf = pf.shape[1]; # Number of Real Power Flow measurements..
    nqf = qf.shape[1]; # Number of Reactive Power Flow measurements..

    h1 = np.mat(V[int(fbus[vi])-1]);
    h2 = np.mat(np.zeros((npi,1)));
    h3 = np.mat(np.zeros((nqi,1)));
    h4 = np.mat(np.zeros((npf,1)));
    h5 = np.mat(np.zeros(((nqf,1))));
    

    for i in range(0,npi):
        m = int(fbus[(ppi[0,i])])-1
        for k in range(0,nbus):
            h2[i] += V[m]*V[k]*(G[m,k]*np.cos(del1[m]-del1[k]) + B[m,k]*np.sin(del1[m]-del1[k]))

    for i in range(0,nqi):
        m = int(fbus[(qi[0,i])])-1
        for k in range(0,nbus):
            h3[i] += V[m]*V[k]*((G[m,k]*np.sin(del1[m]-del1[k])) - (B[m,k]*np.cos(del1[m]-del1[k])))

    for i in range(0,npf):
        m = int(fbus[(pf[0,i])])-1
        n = int(tbus[(pf[0,i])])-1
        h4[i] = -(V[m]**2)*G[m,n] - V[m]*V[n]*(-G[m,n]*np.cos(del1[m]-del1[n]) - B[m,n]*np.sin(del1[m]-del1[n]))

    for i in range(0,nqf):
        m = int(fbus[(qf[0,i])])-1
        n = int(tbus[(qf[0,i])])-1
        h5[i] = -(V[m]**2)*(-B[m,n]+bpq[m,n]) - V[m]*V[n]*(-G[m,n]*np.sin(del1[m]-del1[n]) + B[m,n]*np.cos(del1[m]-del1[n]))

    h111=np.mat(np.concatenate((h1,h2,h3,h4,h5)))
    h= np.zeros(21)
    for i1 in range (21):
        h[i1]=h111[i1]
    return h

In [17]:
def fx(x, dt):
    
    F=np.identity(13)
    FdotX=np.dot(F, x)
    return FdotX

In [18]:
Pi=pd.read_csv("Data/newly_modified.csv")
# Pi=pd.read_csv("data/PQ_fault_modified2.csv")
nbus=7;
Pi= Pi.to_numpy();
num=7




dim_x = 13 # state dimension
dim_z = 21 # measurement dimension
zdata = zdatas(0)




points = MerweScaledSigmaPoints(n=dim_x, alpha=0.8, beta=0.5, kappa=1.0)

#need to improve on the values of alpha beta kappa for better results of z-h rest is good
#now z-h is good but alpha beta kappa can be improved



kf = UnscentedKalmanFilter(dim_x=dim_x, dim_z=dim_z, dt=0.0001, fx=fx, hx=Hx, points=points)
initial_state = np.ones(13)  # Initial state vector
for i in range(6,13):
    initial_state[i]=1
kf.x=initial_state

#original best data to use
# kf.P = np.eye(dim_x)*1e-3# initial state covariance matrix
# kf.Q = np.eye(dim_x)*1e-4 # process noise covariance matrix
# kf.R =np.eye(dim_z)*1e-8



kf.P = np.eye(dim_x)*1e-3# initial state covariance matrix
kf.Q = np.eye(dim_x)*1e-6 # process noise covariance matrix
kf.R =np.eye(dim_z)*1e-8
# for i1 in range(41):
#     y=zdata[i1,5]
#     kf.R[i1][i1]=
c=0
c1=0
#doing for bus wise results
x_g=[]
y_g=[]
# df=pd.read_csv("data/validation_states1.csv")
# df=pd.read_csv("data/Vrms_fault2.csv")
# y_actual=[]
# y_error=[]
# y_per_error=[]
# for i2 in range(14):
#     y_g.append([])
#     y_error.append([])
#     y_actual.append([])
#     y_per_error.append([])
for i in range (1,20):
    # h1=Hx(kf.x)
    zdata = zdatas(i)
    z1 = zdata[:,2]
    z= np.zeros(21)
    for i1 in range (21):
        z[i1]=z1[i1]
    kf.predict() # predict step
    kf.update(z) # update step
    print("state",kf.x)
    

     # ++++++++++++++++++++++
    # Presenting the results
    # ===================


#     c+=1
#     flag=False
#     if(c>0):
#         for i1 in range(13,27):
#             if(i1==19):
#                 continue
#             y_g[i1-13].append(kf.x[i1])
#             diff=(df[f'Subsystem #1|CTLs|Vars|Vrms_{i1-13+1}'][i]/11)-y_g[i1-13][c1]
#             y_error[i1-13].append(diff)
#             y_actual[i1-13].append(df[f'Subsystem #1|CTLs|Vars|Vrms_{i1-13+1}'][i]/11)
#             abs_val=abs(diff)
#             abs_val*=100
#             if(abs_val>10 or flag):
#                 flag=True
#                 abs_val/=18
#             y_per_error[i1-13].append((abs_val))
            
        
#         x_g.append(i)
#         # y_actual.append(df['Subsystem #1|CTLs|Vars|Vrms_1'][0]/11.1)
#         c1+=1
        
        
#         # plot.plot(range(1,nbus+1),kf.x[13:],label=f'{i}')
#     # if(i%1000==0):
#     # plot.title('Voltage Magnitude')
#     # plot.xlabel('Bus number')
#     # plot.ylabel('Magnitude (p.u.)')
#     # plot.xlim(1,nbus)
#     # plot.show()  
    


#     # ++++++++++++++++++++++
#     # end of the results
#     # ===================
# for i2 in range(14):
#     # plot.plot(x_g,y_g[i2],label=f'bus {i2}')
#     if(i2==6):
#         continue
#     plot.plot(x_g,y_error[i2],label=f"error")
#     ax=plot.gca()
#     # ax.set_ylim(-0.00537,-0.005365)
#     plot.title(f'error in bus{i2+1} using ukf')
#     plot.xlabel('time')
#     plot.ylabel('error')
#     plot.show()

#     plot.plot(x_g,y_per_error[i2],label=f"percentage error")
#     plot.title(f'performance index of bus{i2+1} using ukf')
#     ax=plot.gca()
#     ax.set_ylim(0,2)
#     plot.xlabel('time')
#     plot.ylabel('performance index')
#     plot.show()


#     plot.plot(x_g,y_actual[i2],label=f"actual")
#     plot.plot(x_g,y_g[i2],label=f"predicted")
#     ax=plot.gca()
#     # ax.set_ylim(0.93,1.07)
#     plot.title(f'actual and predicted in bus{i2+1} using ukf')
#     plot.xlabel('time')
#     plot.ylabel('voltage magnitude')
#     plot.legend()
#     plot.show()
# for i2 in range(14):
#     plot.plot(x_g,y_g[i2],label=f'bus {i2}')
# plot.plot(x_g,y_g[0],label="measured")
# plot.plot(x_g,y_actual,label="actual")
# plot.title('voltage of bus1 with time')
# plot.xlabel('time')
# plot.ylabel('voltage')
# plot.legend(bbox_to_anchor=(1.05,1),loc='upper left', borderaxespad=0)
# plot.show()
# plot.title('Voltage Magnitude')
# plot.xlabel('Bus number')
# plot.ylabel('Magnitude (p.u.)')
# plot.xlim(1,nbus)
# plot.show()  
    


state [0.22644633 0.22974778 0.23371268 0.23774164 0.23092694 0.23134415
 0.99215718 0.55740157 0.56357694 0.55804668 0.56441311 0.56541858
 0.56075249]
state [-0.09126693 -0.08535963 -0.07852369 -0.07152247 -0.08361064 -0.0827644
  0.98964458  0.95625394  0.95263375  0.95921069  0.94398435  0.94901275
  0.95983064]
state [0.04378248 0.04716378 0.05128485 0.05534821 0.04821361 0.04876466
 0.98885942 0.96595193 0.96735005 0.96683291 0.96321002 0.96601658
 0.96909948]
state [0.04055431 0.04386836 0.04794465 0.0519219  0.04480524 0.04543026
 0.99036985 0.97762081 0.97898604 0.97857746 0.97509145 0.97779541
 0.98080243]
state [0.0390622  0.04231144 0.04635113 0.05017209 0.04311667 0.04382444
 0.99062363 0.97920168 0.9805948  0.98018983 0.97693634 0.97953018
 0.98243091]
state [0.03742813 0.04061344 0.04462439 0.04826925 0.04127541 0.0420791
 0.99033677 0.98006176 0.98147626 0.98106935 0.97803671 0.98052491
 0.98333247]
state [0.03566356 0.03878307 0.04276678 0.04621473 0.0392909  0.0401993